# 基于大模型的广告检索

## 1. 数据集说明
```
|--fusai-2024-cti-bigmodel-retrieval-data
| |--train_data.csv       -> 训练数据集
| |--dev_data.csv         -> 验证数据集
| |--ad_landingpage.txt   -> 广告落地页特征
| |--ad_core_term.txt	-> 广告核心词特征
```


1. train_data.csv / dev_data 训练/测试数据集

    每一行数据对应点击行为的聚合，数据列间用 \t 分割

    | 字段  |    说明    |
    | :---: | :--------: |
    |   q   | 网民搜索词 |
    | ad_id |  广告 ID   |
    | click |  点击次数  |
    
2. ad_landingpage.txt 广告落地页特征

   |    字段    |                 说明                 |
   | :--------: | :----------------------------------: |
   |   ad_id    |               广告 ID                |
   | lp_content | 落地页内容，不同段落间用\<block>连接 |

3. ad_core_term.txt 广告核心词特征(基于百度商业积累，对广告投放页面提取的业务关键字)

   |    字段    |                 说明                 |
   | :--------: | :----------------------------------: |
   |   ad_id    |               广告 ID                |
   | core_terms | 核心词内容，不同核心词间用\<sep>连接 |


## 2. 代码结构
```bash
|--work
| |--data            -> 数据
| |--GenRetr      -> 生成式检索训练代码
| |--scripts         -> 数据准备
| |--submit_templates   -> 模型提交模板
| | |--GenRetr_submit    -> 生成式检索提交模板
|--main.ipynb



In [2]:
## 解压数据集
!tar -zxvf /home/aistudio/data/data274453/fusai-2024-cti-bigmodel-retrieval-data.tar.gz 

# -C /home/aistudio/work/data/

fusai-2024-cti-bigmodel-retrieval-data/
fusai-2024-cti-bigmodel-retrieval-data/train_data.csv
fusai-2024-cti-bigmodel-retrieval-data/ad_landingpage.txt
fusai-2024-cti-bigmodel-retrieval-data/ad_core_term.txt
fusai-2024-cti-bigmodel-retrieval-data/dev_data.csv


In [13]:
## 数据准备(fusai_query)
!python /home/aistudio/work/scripts/prepare_data_gen_ret.py

load train data done
write train json done
load dev data done
write dev json done


In [11]:
## 数据准备(fusai_core_terms_to_query)
!python /home/aistudio/work/scripts/core_terms_to_query.py

JSON objects have been written to /home/aistudio/work/data/fusai_core_terms_to_query.json.


In [ ]:
## 数据准备(fusai_cleaned_ollama_1.json,fusai_cleaned_ollama_all.json介绍)
## A. 利用ollama框架进行数据合成
## 1. 根据广告内容总结生成精简版的广告，得到 train_indexing_qwen_merge.json
!python /home/aistudio/work/scripts/ollama/test_ollama_summarization.py
## 2. 按顺序执行/home/aistudio/work/scripts/ollama/wash.ipynb，对数据做一定的清洗,得到 train_indexing_qwen_merge_cleaned.json

## 3. 按顺序执行/home/aistudio/work/scripts/ollama/handle_data.ipynb,对ollama数据做筛选,得到180w fusai_cleaned_ollama_1.json

## 4. 全部tgt进行增强
!python /home/aistudio/work/scripts/ollama/test_ollama_group_all.py
## 5. 对高点击率和对应src数较少的tgt进行增强
### ollama qwen2:7b 组增强
!python /home/aistudio/work/scripts/ollama/test_ollama_group.py
### ollama qwen2:1.8b 组增强 
!python /home/aistudio/work/scripts/ollama/test_ollama_group_qwen1.8.py
### ollama gemma 2b 组增强 
!python /home/aistudio/work/scripts/ollama/test_ollama_group_gemma2.py

## 最终对上面涉及的数据进行合并，得到全部的ollama数据 fusai_cleaned_ollama_all.json


In [ ]:
## 模型训练
##第一阶段
## 原始数据与core_terms_to_query 1:1 训练 
!bash work/GenRetr/scripts/train_2_files_stage1.sh
##第二阶段
## 原始数据与fusai_cleaned_ollama_1拼接，与core_terms_to_query 1:1 训练 
!bash work/GenRetr/scripts/train_2_files.sh
##第三阶段
## 原始数据与fusai_cleaned_ollama_all与core_terms_to_query 1:1:1 训练 
!bash work/GenRetr/scripts/train_3_files.sh